In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CategoricalColorMapper, ColumnDataSource, CustomJS, HoverTool, RangeSlider
from bokeh.palettes import Viridis256, linear_palette
from bokeh.plotting import figure
from pandas import options, read_csv, to_datetime
from plotly.colors import qualitative

In [ ]:
options.display.max_rows = None
options.display.max_columns = None
options.display.max_colwidth = None
options.display.max_seq_items = None

In [ ]:
data = read_csv(
    "data/region_08.csv",
)
data.head()

## Fixing Missing Dates
- Back Fill if approx date is not valid. (As data is ordered by date.)
- Fill with approx date if valid.

In [ ]:
data["event_date"] = to_datetime(
    data[["iyear", "imonth", "iday"]].rename(columns={"iyear": "year", "imonth": "month", "iday": "day"}),
    errors="coerce",
)
approx_dates1 = to_datetime(data["approxdate"], errors="coerce", format="%m/%d/%Y")
approx_dates2 = to_datetime(data["approxdate"], errors="coerce", format="%B %d, %Y")
approx_dates3 = to_datetime(data["approxdate"], errors="coerce", format="%Y-%m-%d %H:%M:%S")

data["event_date"] = data[approx_dates1.isnull() & approx_dates2.isnull() & approx_dates3.isnull()][
    "event_date"
].bfill()

data = data.drop(["iyear", "imonth", "iday"], axis=1)

data["event_date"] = data["event_date"].fillna(approx_dates1)
data["event_date"] = data["event_date"].fillna(approx_dates2)
data["event_date"] = data["event_date"].fillna(approx_dates3)

data["year"] = data["event_date"].dt.year

## Plotly: Attack Types area graph

In [ ]:
df = data
attack_types_df = pd.melt(
    df,
    id_vars=["eventid", "year", "country_txt", "latitude", "longitude", "nkill", "nkillter", "nwound", "nwoundte"],
    value_vars=["attacktype1_txt", "attacktype2_txt", "attacktype3_txt"],
    var_name="attack_column",
    value_name="attack_type",
)

attack_types_df = attack_types_df.dropna(subset=["attack_type"])

In [ ]:
unique_attacktypes = np.sort(df["attacktype1_txt"].unique())
unique_years = np.sort(df["year"].unique())

multi_index = pd.MultiIndex.from_product(
    [unique_attacktypes, unique_years],
    names=["attacktype1_txt", "year"],
)

In [ ]:
# 3. Attack Types by Year
yearly_distribution = df.groupby(["attacktype1_txt", "year"]).size().reset_index(name="count")
yearly_distribution.head()

In [ ]:
complete_yearly_distribution = (
    yearly_distribution
    .set_index(["attacktype1_txt", "year"])
    .reindex(multi_index, fill_value=0)
    .reset_index()
)

In [ ]:
complete_yearly_distribution.loc[ (complete_yearly_distribution["year"] == 2009)]

In [ ]:
colors = qualitative.D3

In [ ]:
fig = go.Figure()
for idx, atype in enumerate(unique_attacktypes):
    color = colors[idx % len(colors)]
    y = complete_yearly_distribution.loc[complete_yearly_distribution["attacktype1_txt"] == atype, "count"]
    fig.add_trace(go.Scatter(
        x = unique_years,
        y=y,
        mode="lines",
        line={"width": 0.5, "color": color},
        stackgroup="one",
        name=atype,
        visible=True,
    ))

    fig.add_trace(go.Scatter(
        x = unique_years,
        y=y,
        mode="lines",
        line={"width": 0.5, "color": color},
        # stackgroup="one",
        fill="tonexty",
        name=atype,
        visible=False,
    ))

    fig.add_trace(go.Scatter(
        x = unique_years,
        y=y,
        mode="lines",
        line={"width": 0.5, "color": color},
        stackgroup="one",
        # fill="tonexty",
        groupnorm="percent",
        name=atype,
        visible=False,
    ))

fig.update_layout(
    updatemenus=[
        {
            "active" : 0,
            "buttons": [{
                "label": "Stacked",
                "method": "update",
                "args": [{"visible": [idx % 3 == 0 for idx in range(3*len(unique_attacktypes))]}],
            },
            {
                "label": "Grouped",
                "method": "update",
                "args": [{"visible": [idx % 3 == 1 for idx in range(3*len(unique_attacktypes))]}],
            },
            {
                "label": "100% Stacked",
                "method": "update",
                "args": [{"visible": [idx % 3 == 2 for idx in range(3*len(unique_attacktypes))]}],
            }],
            "xanchor":"left",
            "yanchor": "bottom",
            "pad":{"r": 10, "b": 10},
        },
    ],
)

fig.show()

## Bokeh: Target Types and Casualties

In [ ]:
df = data[["year", "targtype1_txt", "nkill", "nwound"]].fillna(0)
df["total"] = df["nkill"] + df["nwound"]

df = df.groupby(["year", "targtype1_txt"], as_index=False).sum()

In [ ]:
output_notebook()

In [ ]:
target_types = sorted(df["targtype1_txt"].unique())
len(target_types)

In [ ]:
df_initial = df.groupby(["targtype1_txt"], as_index=False).sum()

min_size = 10
max_size = 30
min_total = df_initial["total"].min()
max_total = df_initial["total"].max()

if max_total - min_total > 0:
    df_initial["size"] = min_size + (df_initial["total"] - min_total) / (max_total - min_total) * (max_size - min_size)
else:
    df_initial["size"] = min_size

In [ ]:
# Define color mapping
palette = linear_palette(Viridis256, len(target_types))
color_mapper = CategoricalColorMapper(factors=target_types, palette=palette)

# Create data sources
source = ColumnDataSource(df)  # Full yearly data
plot_source = ColumnDataSource(df_initial)  # Aggregated data for the plot

# Create the scatter plot
p = figure(height=800, width=1000,
           title="Casualties by Target Type",
           x_axis_label="Number Killed",
           y_axis_label="Number Wounded",
           tools="pan,wheel_zoom,box_zoom,reset")

scatter = p.scatter("nkill", "nwound", source=plot_source,
                    size="size",
                    color={"field": "targtype1_txt", "transform": color_mapper},
                    legend_field="targtype1_txt", fill_alpha=0.6)

# Add hover tool
hover = HoverTool(tooltips=[
    ("Target", "@targtype1_txt"),
    ("Killed", "@nkill"),
    ("Wounded", "@nwound"),
])
p.add_tools(hover)

# Configure legend
p.legend.title = "Target Types"
p.legend.location = "top_right"
p.legend.click_policy = "hide"

# Create year range slider
year_slider = RangeSlider(start=int(df["year"].min()),
                          end=int(df["year"].max()),
                          value=(int(df["year"].min()), int(df["year"].max())),
                          step=1,
                          title="Year Range")

# Define CustomJS callback for aggregation and size recalculation
# Corrected callback code
callback = CustomJS(args={
    "source": source,
    "slider": year_slider,
    "plot_source": plot_source,
    "all_target_types": target_types,
    "min_size": min_size,
    "max_size": max_size,
}, code="""
    const data = source.data;
    const start = slider.value[0];
    const end = slider.value[1];

    let sums = {};
    for (let tt of all_target_types) {
        sums[tt] = {nkill: 0, nwound: 0, total: 0};
    }

    const year = data['year'];
    const targtype = data['targtype1_txt'];
    const nkill = data['nkill'];
    const nwound = data['nwound'];
    const total = data['total'];
    for (let i = 0; i < year.length; i++) {
        if (year[i] >= start && year[i] <= end) {
            let tt = targtype[i];
            if (sums[tt]) {
                sums[tt].nkill += nkill[i];
                sums[tt].nwound += nwound[i];
                sums[tt].total += total[i];
            }
        }
    }

    let new_targtype = [];
    let new_nkill = [];
    let new_nwound = [];
    let new_total = [];
    let totals = [];
    for (let tt of all_target_types) {
        new_targtype.push(tt);
        new_nkill.push(sums[tt].nkill);
        new_nwound.push(sums[tt].nwound);
        new_total.push(sums[tt].total);
        totals.push(sums[tt].total);
    }

    let min_total = Math.min(...totals);
    let max_total = Math.max(...totals);

    let new_size = [];
    if (max_total - min_total > 0) {
        for (let t of totals) {
            let size = min_size + (t - min_total) / (max_total - min_total) * (max_size - min_size);
            new_size.push(size);
        }
    } else {
        new_size = Array(totals.length).fill(min_size);
    }

    plot_source.data = {
        'targtype1_txt': new_targtype,
        'nkill': new_nkill,
        'nwound': new_nwound,
        'total': new_total,
        'size': new_size
    };
    plot_source.change.emit();
""")

# Attach the callback to the slider
year_slider.js_on_change("value", callback)

# Create layout and display
layout = column(p, year_slider)
# output_file("terrorism_scatter.html")
show(layout)